In [31]:
import pandas as pd

import os
from sklearn.preprocessing import MinMaxScaler
from statsmodels.formula.api import ols

import pandas as pd 
import numpy as np
import statsmodels.api as sm


csv_list=[]
file =os.listdir()
for i in file:
    if ( i[-3:]=='csv'):
        csv_list.append(i)


In [34]:
def make_ols_model(csv_list):
    for i in csv_list:
        df = pd.read_csv(i,encoding='euc-kr')      
        #회귀분석을 위한 표준화 , 변수 선택
        scaler = MinMaxScaler()        
        df_minmax_scaled = scaler.fit_transform(df.iloc[:,[5,6,7,8,9,11]])
        df_minmax_scaled = pd.DataFrame(df_minmax_scaled,columns=df.iloc[:,[5,6,7,8,9,11]].columns)
        cob = (df[['폐업수 합']].max()-df[['폐업수 합']]) / ( df[['폐업수 합']].max()-df[['폐업수 합']].min() )
        data_set = pd.concat([df_minmax_scaled,cob],axis=1)

        x_data = data_set.iloc[:,[0,1,2,3,5,6]] #변수 여러개
        target = data_set.iloc[:,4]

        # for b0, 상수항 추가
        x_data1 = sm.add_constant(x_data, has_constant = "add")

        # OLS 검정
        multi_model = sm.OLS(target, x_data1)
        fitted_multi_model = multi_model.fit()
        fitted_multi_model.summary()

        data_reg = pd.DataFrame([list(fitted_multi_model.params[1:])],columns=[fitted_multi_model.params.index[1:]])
        data_set=data_set.iloc[:,[0,1,2,3,5,6]]
        #가중치
        data_reg=data_reg* (fitted_multi_model.rsquared_adj)
        for j in range(len(data_reg.columns)):
            data_set.iloc[:,j] = data_set.iloc[:,j] * abs(data_reg.iloc[0,j])
        data_set.iloc[:,-1] = data_set.iloc[:,-1] * (1-fitted_multi_model.rsquared_adj)
        data_set['score'+i[7:13]] = data_set.sum(axis=1)
        #최종데이터셋
        data = pd.concat([df.iloc[:,:6],data_set],axis=1)
        print(i[4:10]+"_adj R_squared 값",fitted_multi_model.rsquared_adj)
        #print("###가중치###",data_reg)
        data.to_csv('회귀분석/OLS_'+i,encoding='euc-kr',index=False)
    
    

In [35]:
make_ols_model(csv_list)

201712_adj R_squared 값 0.18630889889584779
201806_adj R_squared 값 0.18443174615606983
201812_adj R_squared 값 0.17992734696602064
201906_adj R_squared 값 0.13453485219696704
201912_adj R_squared 값 0.10695662505516179
202006_adj R_squared 값 0.07456048103692026
202012_adj R_squared 값 0.06390056572451408
202106_adj R_squared 값 0.050351483730648905
202112_adj R_squared 값 0.047668890824880905
202206_adj R_squared 값 0.04371094640131823


KeyError: "None of [Index(['폐업수 합'], dtype='object')] are in the [columns]"

In [323]:
fa_list = os.listdir('회귀분석/')

sum_fa_list = pd.DataFrame()
sum_fa_list= pd.read_csv('회귀분석/'+fa_list[0],encoding='euc-kr').iloc[:,:6]
for i in fa_list:
    df=pd.read_csv('회귀분석/'+i,encoding='euc-kr')
    sum_fa_list = pd.concat([sum_fa_list,df[df.columns[-1]]],axis=1)
    
sum_fa_list.to_csv('회귀분석/sum_ols_list.csv',encoding='euc-kr',index=False)

# 라인별 실행

In [12]:
df.head(2)

,WKT,시도,SIGUNGU_NM,ADM_DR_NM,시군구코드,strc_1806,pop_1806,price_1806,상가수,매출금액 평균,매출액_시점 개수,유동인구 평균,폐업수 합
0,MULTIPOLYGON (((956144.676799997 1926791.06500...,경기도,수원시 장안구,파장동,41111560.0,1675.0,17908.0,255533.41,1082,2.945933e+10,6,4.874069e+06,108
1,"MULTIPOLYGON (((953430.847600002 1923415.439,9...",경기도,수원시 장안구,율천동,41111566.0,1348.0,29536.0,696072.61,1423,2.694915e+10,6,6.958300e+06,127


In [15]:
df.iloc[:,[5,6,7,8,9,11]]

,strc_1806,pop_1806,price_1806,상가수,매출금액 평균,유동인구 평균
0,1675.0,17908.0,255533.41,1082,2.945933e+10,4.874069e+06
1,1348.0,29536.0,696072.61,1423,2.694915e+10,6.958300e+06
2,2775.0,91389.0,1338767.40,839,9.596450e+09,3.166847e+06
3,2775.0,91389.0,1338767.40,1238,1.619929e+10,4.405150e+06
4,2580.0,21409.0,1203498.42,1739,2.446899e+10,4.708194e+06
...,...,...,...,...,...,...
543,2023.0,3660.0,8085.78,178,2.643275e+09,6.178669e+05
544,2370.0,4593.0,11644.36,164,7.241457e+09,9.740883e+05
545,3532.0,6621.0,18914.44,185,3.139956e+09,1.378123e+06
546,5052.0,16985.0,26107.49,1227,1.202932e+10,3.342939e+06


In [16]:
df = pd.read_csv(csv_list[1],encoding='euc-kr')

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_minmax_scaled = scaler.fit_transform(df.iloc[:,[5,6,7,8,9,11]])
df_minmax_scaled = pd.DataFrame(df_minmax_scaled,columns=df.iloc[:,[5,6,7,8,9,11]].columns)
cob = (df[['폐업수 합']].max()-df[['폐업수 합']]) / ( df[['폐업수 합']].max()-df[['폐업수 합']].min() )
data_set = pd.concat([df_minmax_scaled,cob],axis=1)

In [17]:
data_set.head(2)

,strc_1806,pop_1806,price_1806,상가수,매출금액 평균,유동인구 평균,폐업수 합
0,0.155616,0.108341,0.067051,0.194990,0.037184,0.167073,0.744681
1,0.124906,0.178783,0.184365,0.256443,0.034015,0.239237,0.699764


In [18]:
from statsmodels.formula.api import ols

import pandas as pd 
import numpy as np
import statsmodels.api as sm


#다중 선형회귀분석
x_data = data_set.iloc[:,[0,1,2,3,5,6]] #변수 여러개
target = data_set.iloc[:,4]

# for b0, 상수항 추가
x_data1 = sm.add_constant(x_data, has_constant = "add")

# OLS 검정
multi_model = sm.OLS(target, x_data1)
fitted_multi_model = multi_model.fit()
fitted_multi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                매출금액 평균   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     21.62
Date:                Thu, 24 Nov 2022   Prob (F-statistic):           8.10e-23
Time:                        15:35:44   Log-Likelihood:                 902.33
No. Observations:                 548   AIC:                            -1791.
Df Residuals:                     541   BIC:                            -1761.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0599      0.033      1.824      0.069      -0.005       0.124
strc_1806      0.0027      0.012      0.217      0.828      -0.021       0.027
pop_1806      -0.0171      0.013     -1.293      0.196      -0.043       0.009
price_1806     0.0452      0.014      3.124      0.002       0.017       0.074
상가수           -0.0324      0.036     -0.892      0.373      -0.104       0.039
유동인구 평균        0.1591      0.028      5.592      0.000       0.103       0.215
폐업수 합         -0.0617      0.033     -1.889      0.059      -0.126       0.002
==============================================================================
Omnibus:                     1141.046   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2102170.468
Skew:                          15.410   Prob(JB):                         0.00
Kurtosis:                     304.854   Cond. No.                         38.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
#소수점 10자리까지나타나게
pd.options.display.float_format = '{:10f}'.format
#데이터셋 확인
fitted_multi_model.params.index[1:]

Index(['strc_1806', 'pop_1806', 'price_1806', '상가수', '유동인구 평균', '폐업수 합'], dtype='object')

In [20]:
data_reg = pd.DataFrame([list(fitted_multi_model.params[1:])],columns=[fitted_multi_model.params.index[1:]])
data_set=data_set.iloc[:,[0,1,2,3,5,6]]
data_reg=data_reg*fitted_multi_model.rsquared
for i in range(len(data_reg.columns)):
    data_set.iloc[:,i] = data_set.iloc[:,i] * abs(data_reg.iloc[0,i])

In [21]:
#매출액 가중치
data_set.iloc[:,-1] = data_set.iloc[:,-1] * (1-fitted_multi_model.rsquared)

In [22]:
#모델설명력
fitted_multi_model.rsquared

0.1933776502201714

In [23]:
data_set['score'] = data_set.sum(axis=1)

In [24]:
data = pd.concat([df.iloc[:,:6],data_set],axis=1)

In [25]:
data

,WKT,시도,SIGUNGU_NM,ADM_DR_NM,시군구코드,strc_1806,strc_1806,pop_1806,price_1806,상가수,유동인구 평균,폐업수 합,score
0,MULTIPOLYGON (((956144.676799997 1926791.06500...,경기도,수원시 장안구,파장동,41111560.000000,1675.000000,0.000080,0.000359,0.000586,0.001220,0.005139,0.007165,0.014550
1,"MULTIPOLYGON (((953430.847600002 1923415.439,9...",경기도,수원시 장안구,율천동,41111566.000000,1348.000000,0.000064,0.000592,0.001612,0.001605,0.007359,0.006733,0.017965
2,"MULTIPOLYGON (((954598.361899999 1923627.5777,...",경기도,수원시 장안구,정자1동,41111571.000000,2775.000000,0.000133,0.001833,0.003109,0.000946,0.003321,0.008461,0.017804
3,"MULTIPOLYGON (((955513.584699999 1922196.6204,...",경기도,수원시 장안구,정자2동,41111572.000000,2775.000000,0.000133,0.001833,0.003109,0.001396,0.004640,0.007574,0.018685
4,"MULTIPOLYGON (((958087.7235 1922275.02,958080....",경기도,수원시 장안구,영화동,41111580.000000,2580.000000,0.000124,0.000429,0.002794,0.001962,0.004962,0.006437,0.016708
...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,"MULTIPOLYGON (((1022538.551 1954630.9803,10225...",경기도,양평군,청운면,41830370.000000,2023.000000,0.000097,0.000073,0.000010,0.000201,0.000606,0.009326,0.010313
544,"MULTIPOLYGON (((1018405.9698 1946478.1428,1018...",경기도,양평군,양동면,41830380.000000,2370.000000,0.000114,0.000092,0.000018,0.000185,0.000986,0.009462,0.010856
545,"MULTIPOLYGON (((1011829.8535 1945324.8923,1011...",경기도,양평군,지평면,41830395.000000,3532.000000,0.000170,0.000132,0.000035,0.000209,0.001416,0.009417,0.011379
546,"MULTIPOLYGON (((1009252.2599 1953485.9096,1009...",경기도,양평군,용문면,41830400.000000,5052.000000,0.000243,0.000340,0.000052,0.001384,0.003508,0.007711,0.013239
